In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv('/content/drive/MyDrive/DTU/Computational Tools for Data Science/project/dataframeset/df_with_user_id.csv')
len(df)

34376

# Data preprocessing - creating user, movie id

In [ ]:
# user_list = df['username'].tolist()
# print('including duplicated:{}'.format(len(user_list)))

# user_list_c = set(user_list)
# user_list_c = list(user_list_c)

# print('excluding duplicated:{}'.format(len(user_list_c)))

# dict_user = {}

# for i in range(len(user_list_c)):
#   dict_user[user_list_c[i]] = i+1
# print(dict_user)

In [ ]:
# df_c = df.copy()
# df_c['user_id'] = ""

# user_id = []
# for i in range(len(df_c)):
#   user_id.append(dict_user[df_c.iloc[i]['username']])
# print(user_id)
# df_c['user_id'] = user_id
# # df_c

In [ ]:
# # drop duplicated rows
# df_ct = df_c.copy()
# df_ct = df_ct.drop_duplicates(['user_id','movie_id'])
# df_ct = df_ct.reset_index()

In [ ]:
# movie_list_tmp = df_ct['original_title'].tolist()

# # print(len(movie_list_tmp))
# movie_list = set(movie_list_tmp)
# movie_list = list(movie_list)
# # print(len(movie_list))

# dict_movie = {}

# for i in range(len(movie_list)):
#   dict_movie[movie_list[i]] = i+1
# print(dict_movie)

In [ ]:
# df_ct_c = df_ct.copy()
# df_ct_c['new_movie_id'] = ""

# movie_id = []
# for i in range(len(df_ct_c)):
#   movie_id.append(dict_movie[df_ct_c.iloc[i]['original_title']])
# print(movie_id)
# len(movie_id)
# # df_c['user_id'] = user_id

In [ ]:
# df_ct_c['new_movie_id'] = movie_id
# # df_ct_c

In [ ]:
# # remove original movie_id column and rename new movie id as movie_id
# df_ct_c = df_ct_c.drop(columns = ['movie_id'])
# df_ct_c = df_ct_c.rename(columns={'new_movie_id':'movie_id'})
# # df_ct_c

In [ ]:
# df_ct_c.to_csv('/content/drive/MyDrive/DTU/Computational Tools for Data Science/project/dataframeset/df_with_user_id.csv')

# Recommender

In [4]:
df_cp = df.copy()

In [5]:
df_cp = df_cp[['genres', 'original_title',
       'overview', 'runtime', 'vote_average', 'username', 'rating', 'total',
       'date', 'review', 'cast', 'crew', 'user_id', 'movie_id']]

In [6]:
in_use = df_cp[['user_id','movie_id','rating']]
in_use.rating.replace({-1:np.nan},regex=True, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [23]:
piv = in_use.pivot_table(index=['user_id'],columns=['movie_id'],values='rating')

piv_norm = piv.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)),axis=1)
# piv_norm

In [24]:
import scipy as sp
piv_norm.fillna(0,inplace=True)
piv_norm = piv_norm.T
piv_norm = piv_norm.loc[:,(piv_norm!=0).any(axis=0)]

piv_sparse = sp.sparse.csr_matrix(piv_norm.values)
item_similarity = cosine_similarity(piv_sparse)
user_similarity = cosine_similarity(piv_sparse.T)

item_sim_df = pd.DataFrame(item_similarity, index = piv_norm.index, columns=piv_norm.index)
user_sim_df = pd.DataFrame(user_similarity, index = piv_norm.columns,columns=piv_norm.columns)

In [26]:
item_sim_df.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,740,741,742,743,744,745,746,747,748,749
movie_id,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.000000,0.000000,0.000000,0.000000,-0.001108,0.000000,0.000000,0.00000,0.004533,...,0.000000,0.000000,0.000000,0.000000,-0.005806,0.013618,-0.030346,0.078645,0.004354,0.000000
2,0.0,1.000000,0.023009,-0.005062,0.010188,-0.001227,0.000000,-0.001239,0.00000,0.014298,...,0.000000,-0.016652,0.000000,-0.002117,0.000000,0.000000,0.011816,0.000000,0.000000,-0.002437
3,0.0,0.023009,1.000000,0.000516,0.003513,-0.001278,0.001369,-0.001291,-0.00609,0.030477,...,-0.008688,-0.035496,-0.009487,-0.002206,0.000000,-0.000825,0.018699,0.000000,0.000000,-0.010401
4,0.0,-0.005062,0.000516,1.000000,-0.002099,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.0,0.010188,0.003513,-0.002099,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000710


In [27]:
user_sim_df.head()

user_id,20,24,45,46,48,61,66,67,76,82,...,23159,23173,23174,23176,23178,23179,23200,23203,23216,23232
user_id,,,,,,,,,,,,,,,,,,,,,
20,1.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000
24,0.0,1.000000,0.000000,0.0,0.021077,0.0,0.000000,0.0,0.00000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.045566,-0.04572,0.000000,0.065124
45,0.0,0.000000,1.000000,0.0,0.004307,0.0,0.000000,0.0,0.00000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,-0.010401,0.00000,-0.024352,0.045609
46,0.0,0.000000,0.000000,1.0,0.000000,0.0,0.000000,0.0,0.00000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000
48,0.0,0.021077,0.004307,0.0,1.000000,0.0,0.085553,0.0,0.01134,0.012464,...,0.0,0.0,0.0,-0.018815,0.0,0.0,-0.012901,0.00000,-0.005156,0.089038


In [48]:
def top_movies(movie_id):
  cnt = 1
  print("Similar movies to {}:\n".format(movie_id))
  result = item_sim_df.loc[~item_sim_df.index.isin([movie_id]),movie_id].sort_values(ascending=False)[:10]

  for id, score in result.items():
    print('No. {}: {}({:.2f})'.format(cnt,id,score))
    cnt+=1

In [49]:
top_movies(245)

Similar movies to 245:

No. 1: 321(0.10)
No. 2: 262(0.08)
No. 3: 525(0.06)
No. 4: 208(0.06)
No. 5: 131(0.03)
No. 6: 251(0.03)
No. 7: 396(0.03)
No. 8: 330(0.02)
No. 9: 607(0.01)
No. 10: 123(0.01)


In [73]:
import operator
def similar_user_recom(user):
  if user not in piv_norm.columns:
    return('No data available on user {}'.format(user))
  
  # get 10 users by the highest similarity
  sim_users = user_sim_df.sort_values(by=user, ascending=False).index[1:11]
  best = []
  most_common={}

  for i in sim_users:
    # get item list which each user gave highest rate
    # The list should be without the movie which "user" has rated
    result_sorted = piv_norm.loc[:,i][(piv_norm.loc[:,user]==0)].sort_values(ascending=False)
    best.append(result_sorted.index[:5].tolist())
  for j in range(len(best)):
    for k in best[j]:
      if k in most_common:
        most_common[k]+=1
      else:
        most_common[k]=1
  sorted_list = sorted(most_common.items(),key=operator.itemgetter(1),reverse=True)
  return sorted_list[:5]

In [74]:
user_sim_df.columns

Int64Index([   20,    24,    45,    46,    48,    61,    66,    67,    76,
               82,
            ...
            23159, 23173, 23174, 23176, 23178, 23179, 23200, 23203, 23216,
            23232],
           dtype='int64', name='user_id', length=2372)

In [76]:
similar_user_recom(20) # (movie_id, the number of people who have highest user similarity who rated "movie_id"?)

[(495, 9), (494, 8), (1, 7), (503, 5), (502, 3)]

In [60]:
piv_norm

user_id,20,24,45,46,48,61,66,67,76,82,...,23159,23173,23174,23176,23178,23179,23200,23203,23216,23232
movie_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.213501,0.0,0.583333,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.070644,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,-0.072214,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.070644,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
746,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
747,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
